In [ ]:
#| label: vfaFig4jn
from repo2data.repo2data import Repo2Data
import os 
import pickle
import matplotlib.pyplot as plt
import chart_studio.plotly as py
import plotly.graph_objs as go
import numpy as np
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from IPython.display import display, HTML
from plotly import tools
from pathlib import Path

from contextlib import contextmanager
import sys, os

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

with suppress_stdout():
    repo_path = Path(*os.path.normpath(os.getcwd()).split(os.path.sep)[:os.path.normpath(os.getcwd()).split(os.path.sep).index('mooc')+1])
    data_req_path = repo_path / "binder" / "data_requirement.json"
    data_path = repo_path / '..' / "data"
    dataset_path = data_path / "qmrlab-mooc"
    
    if not dataset_path.exists():
        repo2data = Repo2Data(data_req_path)
    
DATA_ROOT = dataset_path / "t1-book-neurolibre"    
filename = os.path.join(DATA_ROOT,"02",'figure_4.pkl')

with open(filename, 'rb') as f:
    params, Nex_range, signal_ideal_spoil, signal_optimal_crush_and_rf_spoil, signal_no_gradient_and_rf_spoil = pickle.load(f)

config={'showLink': False, 'displayModeBar': False}
init_notebook_mode(connected=True)

data1 = [dict(
        visible = False,
        mode = 'lines',
        x = params["EXC_FA"],
        y = abs(np.squeeze(np.asarray(signal_ideal_spoil[ii]))),
        name = 'Ideal Spoiling',
        text = 'Ideal Spoiling',
        hoverinfo = 'x+y+text') for ii in range(len(Nex_range))]

data1[10]['visible'] = True

data2 = [dict(
        visible = False,
        mode = 'lines',
        x = params["EXC_FA"],
        y = abs(np.squeeze(np.asarray(signal_optimal_crush_and_rf_spoil[ii]))),
        name = 'Gradient & RF Spoiling',
        text = 'Gradient & RF Spoiling',
        hoverinfo = 'x+y+text') for ii in range(len(Nex_range))]

data2[10]['visible'] = True

data3 = [dict(
        visible = False,
        mode = 'lines',
        x = params["EXC_FA"],
        y = abs(np.squeeze(np.asarray(signal_no_gradient_and_rf_spoil[ii]))),
        name = 'No Spoiling',
        text = 'No Spoiling',
        hoverinfo = 'x+y+text') for ii in range(len(Nex_range))]

data3[10]['visible'] = True

data = data1 + data2+ data3

steps = []
for i in range(len(Nex_range)):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(data1)],
        label = str(Nex_range[i])
        )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    x = 0,
    y = -0.02,
    active = 10,
    currentvalue = {"prefix": "n<sup>th</sup> TR: <b>"},
    pad = {"t": 50, "b": 10},
    steps = steps
)]

layout = go.Layout(
    width=580,
    height=450,
    margin=go.layout.Margin(
        l=80,
        r=40,
        b=60,
        t=10,
    ),
    annotations=[
        dict(
            x=0.5004254919715793,
            y=-0.18,
            showarrow=False,
            text='Excitation Flip Angle (°)',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=-0.15,
            y=0.5,
            showarrow=False,
            text='Signal',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
    ],
    xaxis=dict(
        autorange=False,
        range=[0, params['EXC_FA'][-1]],
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        autorange=True,
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    legend=dict(
        x=0.5,
        y=0.9,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    ), 
    sliders=sliders,
    plot_bgcolor='white'
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'vfa_fig_4.html', config = config)


